In [ ]:
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt
from PIL import Image, ImageFilter
from scipy import signal
from skimage import measure
from joblib import Parallel, delayed
from paretoset import paretoset
import sklearn.metrics.pairwise as pdist
#import skfmm

import leaf_processing_helpers as lph

Mask out background

In [ ]:
GRASPER_CLEARANCE = 0.02 #cm clearance for bottom of grasper

In [ ]:
leafs = lph.apply_depth_mask("../data/Pointclouds/demo_5.pcd", '../data/Images/aggrigated_masks5.png',
                             "../data/Arrays/depth5.npy", "../data/Images/left_rect5.png",  plot=True)

Clean mask

Small clusters with a pixel area of less than 200 are removed.

In [ ]:
mask = lph.clean_mask(leafs)

leafs[:,:,3] = mask

Perform convolution with needle array to erode away leaf edges

In [ ]:
depth_image = leafs[:,:,2].astype('float32')
mask_image = leafs[:,:,3].astype('uint8')
kernels = lph.get_kernels(depth_image, mask_image)
graspable_mask = lph.compute_graspable_areas(kernels, mask_image)

In [ ]:
binary_grasp_mask = graspable_mask >=1
leafs[:,:,3] = leafs[:,:,3]*binary_grasp_mask
leafs_ = np.reshape(leafs[:,:,0:3], (1555200, 3))
index = np.argwhere(leafs_ == [0,0,0])
inverse_index = np.nonzero(leafs_[:,2])
leafs__ = np.delete(leafs_, index,0)

In [ ]:
processed_pcd = o3d.geometry.PointCloud()
processed_pcd.points = o3d.utility.Vector3dVector(leafs__)

Compute normal vectors

Each leaf's normal vectors are computed, which is then used to estimate how flat each portion of the leaf is.
This is working under the assumption that flat means pointing straight up towards the camera. A more robust implementation
will be needed if flatness is defined relative to each leaf.

In [ ]:
lph.compute_normals(processed_pcd)
sqrt_dist = np.sum((processed_pcd.normals[999])**2, axis=0)
dist = np.sqrt(sqrt_dist)
normal_orientation = np.asarray(processed_pcd.normals)[:,2]
normal_orientation = abs(normal_orientation)
orientation_color = np.zeros((len(normal_orientation), 3))
orientation_color[:,0]=normal_orientation
orientation_color[:,1]=0
orientation_color[:,2]=0
processed_pcd.colors = o3d.utility.Vector3dVector(orientation_color)

In [ ]:
leaf_flatness = np.zeros((1555200, 1))
j=0
for i in range(len(inverse_index[0])):
    current_index = inverse_index[0][i]
    leaf_flatness[current_index,0] = normal_orientation[j]
    j+=1
    
leaf_flatness = np.reshape(leaf_flatness, (1080,1440,1))
leafs = np.concatenate((leafs, leaf_flatness), axis=2)

In [ ]:
fig, ax = plt.subplot_mosaic([
            ['mask', 'z', 'flatness']
        ], figsize=(15,10))

ax['z'].imshow((leafs[:, :, 2]))
ax['z'].set_title("Depth (m)")
ax['mask'].imshow(leafs[:, :, 3])
ax['mask'].set_title("Graspable Area")
ax['flatness'].imshow(leafs[:,:,4])
ax['flatness'].set_title("Flatness Estimation (Normal Z Value)")
plt.show()

Combine graspable area with flatness data to compute best part to grasp on each leaf

In [ ]:
alpha = 0.4
graspable_area = binary_grasp_mask
smooth_section = leafs[:,:,4]
leaf_selection_a = alpha*smooth_section+(1-alpha)*graspable_area


leaf_selection_ab = np.where(leaf_selection_a, leaf_selection_a >= np.amax(leaf_selection_a)*.95,
                                      0)

leafs[:,:,3] = leaf_selection_ab*leafs[:,:,3]
viable_leaf_regions = lph.clean_mask(leafs)


fig, ax = plt.subplot_mosaic([
            ['grasp', 'flatness', 'combined']
        ], figsize=(15,10))

ax['grasp'].imshow(graspable_area)
ax['grasp'].set_title("Graspable area")
ax['combined'].imshow(viable_leaf_regions)
ax['combined'].set_title("Graspable Flat Areas")
ax['flatness'].imshow(leafs[:,:,4])
ax['flatness'].set_title("Flatness Estimation (Normal Z Value)")
ax['grasp'].set_axis_off()
ax['combined'].set_axis_off()
ax['flatness'].set_axis_off()
plt.show()

# Implement function to measure reduction in area from clean up to determine if mask should still exist.

In [ ]:
unique = np.unique(leafs[:,:,3])
depth_list = lph.mean_mask_depth(leafs)
depth_list_norm = lph.mean_mask_depth(leafs, normalized=True)

tall_leaves = lph.find_tall_leaves(depth_list, leafs)
plt.imshow(tall_leaves)
plt.show()


In [ ]:
leafs[:,:,3] = viable_leaf_regions
viable_leaf_regions = lph.clean_mask(leafs)
cleaned_masks = viable_leaf_regions >= 1
cleaned_masks_tall = tall_leaves >= 1
cleaned_masks = np.where(cleaned_masks, cleaned_masks == 0, 1)
cleaned_masks_tall = np.where(cleaned_masks_tall, cleaned_masks_tall == 0, 1)
SDF_X = skfmm.distance(cleaned_masks_tall, dx=1)
SDF = skfmm.distance(cleaned_masks, dx=1)

In [ ]:
fig, ax = plt.subplot_mosaic([
            ['all', 'tall']
        ], figsize=(15,10))

ax['all'].imshow(SDF)
ax['all'].set_title("Full SDF")
ax['tall'].imshow(SDF_X)
ax['tall'].set_title("Tall Leaves SDF")
plt.show()


In [ ]:
min_global = np.unravel_index(SDF.argmin(), SDF.shape)
max_global = np.unravel_index(SDF.argmax(), SDF.shape)
min_tall = np.unravel_index(SDF_X.argmin(), SDF_X.shape)
max_tall = np.unravel_index(SDF_X.argmax(), SDF_X.shape)

In [ ]:

tall_leaves = tall_leaves * viable_leaf_regions
centroids, mask, areas = lph.get_centroids(viable_leaf_regions.astype('uint8'))
centroids_tall, mask_tall, areas_tall = lph.get_centroids(tall_leaves.astype('uint8'))
leafs[:,:,3] = mask
for i, _ in enumerate(centroids_tall):
    plt.scatter(centroids_tall[i][0], centroids_tall[i][1],s=1,c='r')
left_image = Image.open("Images/left_rect5.png")
left_array = np.asarray(left_image)[:, :, 0:3]
plt.imshow(left_array)
plt.title("Estimated Optimal Leaf Grasp Locations")
plt.show()

In [ ]:
data = lph.compute_minmax_dist(centroids, min_global, max_global)
data_tall = lph.compute_minmax_dist(centroids_tall, min_tall, max_tall)

In [ ]:
mask = paretoset(data)
paretoset_sols = data[mask]
res = mask

In [ ]:
mask_tall = paretoset(data_tall)
paretoset_sols_tall = data_tall[mask_tall]
res_tall = mask_tall

In [ ]:
fig, ax = plt.subplot_mosaic([
            ['all', 'tall']
        ], figsize=(15,10))

ax['all'].scatter(data[:, 0], data[:, 1], zorder=10, label="All leaves", s=50, alpha=0.8)
ax['all'].scatter(paretoset_sols[:, 0], paretoset_sols[:, 1], zorder=5, label="optimal leaf", s=150, alpha=1)
ax['all'].set_title("Full SDF Paretoset")
ax['tall'].scatter(data_tall[:, 0], data_tall[:, 1], zorder=10, label="All leaves", s=50, alpha=0.8)
ax['tall'].scatter(paretoset_sols_tall[:, 0], paretoset_sols_tall[:, 1], zorder=5, label="optimal leaf", s=150, alpha=1)
ax['tall'].set_title("Tall Leaves Paretoset")
plt.show()

In [ ]:
opt_leaves = np.where(res==True)[0]
opt_leaves_tall = np.where(res_tall==True)[0]
plt.imshow(left_array)
for i_ in opt_leaves:
    plt.plot(centroids[i_][0], centroids[i_][1], 'r*')
for i_ in opt_leaves_tall:
    plt.plot(centroids[i_][0], centroids[i_][1], 'b*')
plt.show()

Qualifications for best leaf selection

Two cases - one where the leaf is on the outside edge of the plant, and one where the leaf is within the cluster

For case one - outside leaf:

Value height
Value area
Value isolation
Value not being on the very edge of the image

For case two - inside leaf:

Value height - higher weight than outside leaf height. Might need to consider its height difference relative to other leaves around it
value area

In [ ]:
mask_1 = paretoset(centroids, sense=["max", "min"], distinct=True)
mask_2 = paretoset(centroids, sense=["min", "max"], distinct=True)

mask = mask_1+mask_2

areas = (areas - np.min(areas))/np.max(areas)

# If quantity of leaves is low enough, don't make distinction between outside and inside leaves
if len(centroids) < 10:
    mask = np.zeros(len(centroids))
    mask[:] = True

leaf_score = np.zeros(len(centroids))
index = np.unique(leafs[:, :, 3])
for i, _ in enumerate(centroids):
    if i == 0:
        continue
    points = lph.find_number_of_points_in_radius(centroids, centroids[i], 120)
    #leaf_score[i] -= points
    leaf_score[i] += (1-depth_list[i])
    leaf_score[i] += areas[i]
    if leaf_score[i] < 0:
        leaf_score[i] = 0


for i, _ in enumerate(centroids):
    if mask[i] == True:
        plt.scatter(centroids[i][0], centroids[i][1],c='c')
        if leaf_score[i] > 1.2:
            plt.scatter(centroids[i][0], centroids[i][1],c='b')
    else:
        leaf_score[i] -= areas[i]*.1
        if leaf_score[i] > 1.2:
            print(leaf_score[i])
            plt.scatter(centroids[i][0], centroids[i][1],c='b')
        else:
            plt.scatter(centroids[i][0], centroids[i][1],c='r')
left_image = Image.open("Images/left_rect5.png")
left_array = np.asarray(left_image)[:, :, 0:3]
plt.imshow(left_array)
plt.title("Estimated Optimal Leaves")
plt.show()

In [ ]:
pcd = o3d.geometry.PointCloud()
points = np.reshape(leafs[:, :, 0:3], (1555200, 3))
points = np.delete(points, np.argwhere(points == [0, 0, 0]), axis=0)
pcd.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([pcd])